#Query Formation : Group 15

1.   Digvijaysinh Sujaysinh Mane -- 2018A7PS0093G
2.   Nupur Funkwal               -- 2018A7PS0624G
3.   Mitali Doshi                -- 2019A7PS0064G
4.   Nirzari Kalpesh Shah        -- 2019A8PS0576G
5.   Prayas Kumar Singh          -- 2019A8PS0439G



#Files submitted along with the code:
Output files:

*   Output file for *base model* on train data  -- train_result_tri.txt
*   Output file for *base model* on test data   -- test_result_tri.txt
*   Output file for *final model* on train data -- train_file.txt
*   Output file for *final model* on test data. -- test_result.txt


Final presentation









# Importing Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp "/content/drive/MyDrive/NLP/qac_background.txt" "corpus.txt"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#required libraries 
!pip install rank_bm25 nltk
import spacy
import nltk
import math
from nltk.corpus import stopwords
nltk.download("punkt")
nltk.download("stopwords")
import pandas as pd
nlp = spacy.load('en')
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language='english')
from nltk import word_tokenize
from nltk import everygrams
from nltk.util import ngrams
from collections import Counter, defaultdict

# loading data into respective files
training = pd.read_csv('/content/drive/MyDrive/NLP/qac_training.tsv', sep='\t', header = None)
test = pd.read_csv('/content/drive/MyDrive/NLP/qac_test.tsv', sep='\t', header = None)
validation = pd.read_csv('/content/drive/MyDrive/NLP/qac_validation.tsv', sep='\t', header = None)

x = training.loc[:, 0]    #Column A
y = training.loc[:, 1]    #Column B

data = training.loc[:, 1].unique()
for i in range(len(data)):
  data[i]+=(" $")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Part 1: Spelling Correction

Helper Functions


In [ ]:
def tokenize_queries():
  corpus = open("corpus.txt", 'r')
  
  tokenized_queries = list()
  for query in corpus.readlines() :
    tokens = ['<s>']

    for token in query.split():
      tokens.append(token)

    tokens.append('<e>')
    
    tokenized_queries.append(tokens)
  
  return tokenized_queries


def get_token_freqency(tokenized_queries) :
  token_freqency = dict()

  for query in tokenized_queries :
    for token in query:
      if token in token_freqency:
        token_freqency[token] += 1
      else:
        token_freqency.update({ token : 1 })

  return token_freqency


def get_vocab(token_freqency, count_threshold = 10.0, count_limit = float('inf')):

  vocab = set()

  for token, count in token_freqency.items():
    if count > count_threshold and count < count_limit:
      vocab.add(token);

  return vocab


def build_bigram_model(tokenized_queries):
  bigram_model = dict()

  for query in tokenized_queries :
    for i in range(len(query) - 1):
      bigram = query[i] + query[i+1]
      if bigram in bigram_model:
        bigram_model[bigram] += 1
      else:
        bigram_model.update({ bigram : 1 })

  for bigram in bigram_model.keys():
    bigram_model[bigram] = math.log(bigram_model[bigram])

  return bigram_model


## edit distance (levenshtein distance)

def levenshtein_distance(s1, s2):
  
  # 2 - D matrix 'd'
  # x = [[val for i in range(N)] for j in range(M)
  m=len(s1)
  n=len(s2)
  d=[[0 for i in range(0,m+1)]for j in range(0,n+1)]

  for i in range(0, m+1):
    d[0][i]=i
  for j in range(0, n+1):
    d[j][0]=j

  for i in range(1, m+1):
    for j in range(1, n+1):
      if s1[i-1]==s2[j-1]:
        d[j][i]=d[j-1][i-1]
      else:
        d[j][i]=min(d[j-1][i]+1, d[j][i-1]+1, d[j-1][i-1]+1)

  return d[n][m]


def get_query_prob(query, bigram_model):
  prob = 0
  for i in range(len(query) - 1):
    bigram = query[i] + query[i+1]
    if bigram in bigram_model:
      prob += bigram_model[bigram]

  return prob

In [ ]:
tokenized_queries = tokenize_queries()
token_freqency = get_token_freqency(tokenized_queries)
vocab = get_vocab(token_freqency)
bigram_model = build_bigram_model(tokenized_queries)

Correction Functions

In [ ]:
def do_spell_check(incorrect_word, dictionary, max_edit_distance):
  output_list=[]
  for word in dictionary:
    if abs(len(word)-len(incorrect_word))<=max_edit_distance:
      if levenshtein_distance(word, incorrect_word)<=max_edit_distance:
        output_list.append(word)
  
  return output_list


def get_simmilar_words(word, vocab, edit_distance = 2.0, cutoff_factor = 0.9):
  if cutoff_factor > 1 or cutoff_factor < 0:
      raise Exception("cutoff frequency needs to be [0,1]") 

  guesses = do_spell_check(word, vocab, edit_distance)
  max_freq = 0
  for word in guesses:
    if max_freq < token_freqency[word]:
      max_freq = token_freqency[word]

  candidate_words = list()
  for word in guesses:
    if token_freqency[word] >= cutoff_factor * max_freq:
      candidate_words.append(word)

  if '<s>' in candidate_words:
    candidate_words.remove('<s>')
  if '<e>' in candidate_words:
    candidate_words.remove('<e>')

  return candidate_words


def context_based_spell_correction(error_word, word_before, word_after, vocab, bigram_model, edit_distance = 2.0, cutoff_factor = 0.0): 
  candidate_words = get_simmilar_words(error_word, vocab, edit_distance, cutoff_factor)

  max_prob = -1
  suggested_word = ""
  for word in candidate_words:

    prob = 0
    bigram = word_before + word
    if bigram in bigram_model:
      prob1 = bigram_model[bigram]
    else:
      prob1 = 0

    bigram = word + word_after
    if bigram in bigram_model:
      prob2 = bigram_model[bigram]
    else:
      prob2 = 0

    if max_prob < prob1 + prob2:
      max_prob = prob1 + prob2
      suggested_word = word

  return suggested_word


def query_correction(query_str, vocab, bigram_model, edit_distance = 2.0, cutoff_factor = 0.8):
  query = query_str.split()

  query.insert(0, '<s>')
  query.append('<e>')

  #spell check of each word in query
  for i in range(1, len(query) - 1):
    if query[i] not in vocab:
      w = context_based_spell_correction(query[i], query[i-1], query[i+1], vocab, bigram_model, 1, cutoff_factor)
      if not w:
        query[i] = '<unk>'
      else:
        query[i] = w

  #simmilar spelling word check using bigram
  candidate_words = list()
  for word in query[1:-1]:
    candidate_words.append( get_simmilar_words(word, vocab, edit_distance, cutoff_factor ))
  
  corrected_query = query
  max_query_prob = get_query_prob(query, bigram_model)
  for i in range(1, len(query) - 1):
    for word in candidate_words[i - 1]:
      candidate_query = query[0:i] + [word] + query[i+1:]
      query_prob = get_query_prob(candidate_query, bigram_model)

      if max_query_prob < query_prob:
        max_query_prob = query_prob
        corrected_query = candidate_query

  return corrected_query


### spelling correct
* air conditioner *fillers*
* gta san andreas *ceat* codes

### all words of query incorrect
* psycological disordr
* oline spel chcker

### context based word correction
* *whether* in london
* make *monkey* at home

### spelling and context based corrections
* *back* of america visa *crrd*
* what is the *tame* in *jpan*

### limitations
* marry jane watson
* basketball hall of fame

In [ ]:
queries = ['whether in london', 'air conditioner fillers', 'oline spel chcker', 'back of america visa crrd']
for q in queries: 
  print(query_correction(q, vocab, bigram_model, 2, 0.5))

['<s>', 'weather', 'in', 'london', '<e>']
['<s>', 'air', 'conditioner', 'filters', '<e>']
['<s>', 'online', 'spell', 'checker', '<e>']
['<s>', 'bank', 'of', 'america', 'visa', 'card', '<e>']


#Part 2: Query Completion
Trained model using the training data to complete incomplete queries for the qac_training.tsv file



# Preprocessing: On both test and train datasets

In [ ]:
#data == training data
data = training.loc[:, 1].unique()
for i in range(len(data)):
  data[i]+=(" $")

#test_data == test data
test_data = test.loc[:,1].unique()
for i in range(len(test_data)):
  test_data[i]+=(" $")


#the following preprocessing steps were carried out to find the intersection of data between train and testing data
#the following is an instance of utilising Porter's Stemmer, we ended up exploring Snowball Stemmer as well as Lemmatization
#the results were very similar with intersection being close to 1750 words for all three methods

train_stem = []
test_stem = []

for query in data:
  words = word_tokenize(query)
  for w in words:
      if w not in train_stem:
        train_stem.append(stemmer.stem(w))

for query in test_data:
  words = word_tokenize(query)
  for w in words:
      if w not in test_stem:
        test_stem.append(stemmer.stem(w))

#returns intersection of two lists
def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

print("Common tokens in test and training=",len(intersection(train_stem, test_stem)))
print("Total tokens in test=", len(test_stem))



Common tokens in test and training= 1725
Total tokens in test= 4461


In [ ]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

train_lem = []
test_lem = []

test_data = test.loc[:,1].unique()
for i in range(len(test_data)):
  test_data[i]+=(" $")

for query in data:
  words = word_tokenize(query)
  for w in words:
      if w not in train_lem:
        train_lem.append(lemmatizer.lemmatize(w))

for query in test_data:
  words = word_tokenize(query)
  for w in words:
      if w not in test_lem:
        test_lem.append(lemmatizer.lemmatize(w))

#returns intersection of two lists
def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

print('Common tokens in test and training=',len(intersection(train_lem, test_lem)))
print('Total tokens in test=',len(test_lem))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Common tokens in test and training= 1698
Total tokens in test= 4090


#Character Gram Models

In [ ]:
#baseline model: utilises trigram
def char_prediction_trigram(query, k_gram_model,flag):
  #query is to be treated as a string, so we need not make a list of tokens separately
  length=len(query)

  #base case: 2; the string is an empty string
  if (not(length)):
    return [(1,"")]

  #since the function is recursive, this is the base case: query is complete: return probability of the query and the query itself
  if (flag and query[length-1]==' '):
    return [(calculate_char_probability(query), query)]
  if (query[length-1]=='$'):
    return [(calculate_char_probability(query), query)]

  key=[]
  for i in range(length):
    key.append(query[i])

  candidate_tokens=[]
  k=3

  while(len(key)>k-1):
    key.pop(0)

  #recheck
  while (k > 2):
    if tuple(key) not in k_gram_model[k].keys():
      # print(key, end=" ")
      # print(k)
      key.pop(0)
      k-=1
    else:
      candidate_dict = k_gram_model[k][tuple(key)]
      break

  if k==2:
    return [(calculate_char_probability(query), query)]

  resultant_queries = []
  for i in list(candidate_dict.keys()):
    new_query = query + i
    resultant_queries += char_prediction(new_query, k_gram_model,flag)
    
  return Sort_Tuple(resultant_queries)[0:10]


#advanced Character Gram model with back-off and higher k-gram models
#input data
char_data = training.loc[:, 1].unique()
for i in range(len(char_data)):
  char_data[i]+=(" $")

#builds n_gram character models
def n_gram_model_character(n, char_data):
  n_gram_model=defaultdict(lambda: defaultdict(lambda: 0))

  for sentence in char_data:
    l = len(sentence)
    if (l <= n-2):
      continue
    n_grams=list(everygrams(sentence,n,n))
    
    # making key
    for words in n_grams:
      key_list=[]
      for x in range(len(words)-1):
        key_list.append(words[x])
      key_list=tuple(key_list)
      n_gram_model[key_list][words[n-1]]+=1

  for x in n_gram_model:
    total_count = float(sum(n_gram_model[x].values()))
    for w in n_gram_model[x]:
      n_gram_model[x][w] /= total_count

  return n_gram_model


k_gram_model_char=[0,0,0]

for i in range(3,31):
  k_gram_model_char.append(n_gram_model_character(i,char_data))


#helper functions 
import math

#sorts list of tuples
def Sort_Tuple(tup): 
    tup.sort(key = lambda x: -x[0]) 
    return tup 

#calculates frequency for each char in data
def calculate_freq_char(data):
  tokenized_queries = []
  for query in data :
    tokens = []
    for token in query:
      tokens.append(token)
    tokenized_queries.append(tokens)

  tokens_count = {}

  for query in tokenized_queries :
    for token in query:
      if token in tokens_count:
        tokens_count[token] += 1
      else:
        tokens_count.update({ token : 1 })

  return tokens_count


char_count = calculate_freq_char(char_data)


#calculates probability for the entire query
def calculate_char_probability(query_words):
  l=len(query_words)
  total_count = 0
  for token in char_count.keys():
    total_count += char_count[token]

  prob = 10**10
  prob *= (char_count[query_words[0]]/total_count)

  for i in range(2,l):
    prob *= (k_gram_model_char[3][(query_words[i-2],query_words[i-1])][query_words[i]])

  return prob


#char prediction
def char_prediction(query, k_gram_model, flag):
  #query is to be treated as a string, so we need not make a list of tokens separately
  length=len(query)

  #base case: 2; the string is an empty string
  if (not(length)):
    return [(1,"")]

  #since the function is recursive, this is the base case: query is complete: return probability of the query and the query itself
  if (flag and query[length-1]==' '):
    return [(calculate_char_probability(query), query)]
  if (query[length-1]=='$'):
    return [(calculate_char_probability(query), query)]

  
  key=[]
  for i in range(length):
    key.append(query[i])

  candidate_tokens=[]
  k=30
  if length < 29:
    k=length+1

  while(len(key)>k-1):
    key.pop(0)

  #recheck
  while (k > 2):
    if tuple(key) not in k_gram_model[k].keys():
      # print(key, end=" ")
      # print(k)
      key.pop(0)
      k-=1
    else:
      candidate_dict = k_gram_model[k][tuple(key)]
      break

  if k==2:
    return [(calculate_char_probability(query), query)]

  resultant_queries = []
  for i in list(candidate_dict.keys()):
    new_query = query + i
    resultant_queries += char_prediction(new_query, k_gram_model,flag)
    
  return Sort_Tuple(resultant_queries)[0:10]

Helper Functions

In [ ]:
#making a dictionary to complete word
def mapping_from_right(corpus):
  #traversing through the corpus to assist with edit distance
  final_dict={}
  for word in corpus:
    for l in range(len(word)):
      #start will be always 0
      #end will be l+1
      key = word[0:l+1]
      if key in final_dict.keys():
        final_dict[key].append(word)
      else:
        final_dict[key]=[]
        final_dict[key].append(word)
  return final_dict


#helper function to make a bigram model, parameter taken is a list of sentences
#returns a dictionary of dictionaries
#first key is the first word in bigram; the second key the second word in bigram and the value is the frequency of the subsequently formed bigram
def make_bigram(data):
  bigram_model={}

  for sentence in data:
    l = len(sentence.split())
    if (l <= 0):
      continue
    bi_grams=list(everygrams(sentence.split(),2,2))

    for t in bi_grams:
      if (t[0] in bigram_model.keys() and t[1] in bigram_model[t[0]].keys()):
        bigram_model[t[0]][t[1]]+=1
      elif (t[0] not in bigram_model.keys()):
        bigram_model[t[0]]={}
        bigram_model[t[0]][t[1]]=1
      elif (t[1] not in bigram_model[t[0]].keys()):
        bigram_model[t[0]][t[1]]=1
      

  for x in bigram_model:
    total_count = float(sum(bigram_model[x].values()))
    for w in bigram_model[x]:
      bigram_model[x][w] /= total_count
        
  return bigram_model


def n_gram_model(n, data):
  n_gram_model=defaultdict(lambda: defaultdict(lambda: 0))

  for sentence in data:
    l = len(sentence.split())
    if (l <= n-2):
      continue
    n_grams=list(everygrams(sentence.split(),n,n))
    
    # making key
    for words in n_grams:
      key_list=[]
      for x in range(len(words)-1):
        key_list.append(words[x])
      key_list=tuple(key_list)
      n_gram_model[key_list][words[n-1]]+=1

  for x in n_gram_model:
    total_count = float(sum(n_gram_model[x].values()))
    for w in n_gram_model[x]:
      n_gram_model[x][w] /= total_count

  return n_gram_model


#list of k_gram_models
#bigram model will be k_gram_model[2], trigram model will be k_gram_model[3] and so on
k_gram_model=[0,0]
#the function for making a bigram model is entirely different
k_gram_model.append(make_bigram(data))
for i in range(3,10):
  k_gram_model.append(n_gram_model(i,data))


#returns intersection of two lists
def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))


#calculates freuency for each token in data
def calculate_freq(data):
  tokenized_queries = []
  for query in data :
    tokens = []
    for token in query.split():
      tokens.append(token)
    tokenized_queries.append(tokens)

  tokens_count = {}

  for query in tokenized_queries :
    for token in query:
      if token in tokens_count:
        tokens_count[token] += 1
      else:
        tokens_count.update({ token : 1 })

  return tokens_count


#calculates probability for the entire query
def calculate_query_probability(query):
  query_words=query.split()
  l=len(query_words)
  total_count = 0
  for token in tokens_count.keys():
    total_count += tokens_count[token]

  prob = 10**10
  prob *= tokens_count[query_words[0]]/total_count

  for i in range(1,l):
    prob *= k_gram_model[2][query_words[i-1]][query_words[i]]

  return prob


tokens_count = calculate_freq(data)


#returns list of tuple of probabilities and the final queries
#here the flag means that the last word of input is complete or not
def query_prediction(query, k_gram_model, threshold, flag):
  tokens = []
  for token in query.split():
    tokens.append(token)
  
  length = len(tokens)

  if (query[length-1] == "$"):
    return [(calculate_char_probability(query), query)]

  #for incomplete last word in query
  if flag == False:
    resultant_queries = []
    #list of possible correct last words        --> some threshold using edit distance can be set here
    x = mapping_from_right(list(tokens_count.keys()))
    if tokens[length-1] in x.keys():
      possible_words = x[tokens[length-1]]

    #check if these possible words have atleast a bigram relation
    if length> 1 and tokens[length-2] in k_gram_model[2].keys():
      bi_words = list(k_gram_model[2][tokens[length-2]].keys())
      possible_words = intersection(possible_words, bi_words)
    #print('pos=', possible_words)

    for i in possible_words:
      tokens[length-1] = i
      new_query = ''
      for t in tokens:
        new_query += (t+' ')

      new_query = new_query[:-1]
      #print('new query=', new_query)
      resultant_queries += query_prediction(new_query, k_gram_model, threshold, True)
    
    return Sort_Tuple(resultant_queries)[0:10]

  else:
    #getting candidate tokens
    if length > 1:
      key = []
      for i in range(length):
        key.append(tokens[i])
      #key = tuple(key)
    
    else:
      key = (tokens[0])
 
    #base case
    # if length>1:
    #   if tuple(key) not in list(k_gram_model[length+1].keys()):
    #     return [(calculate_query_probability(query), query)]

    if length == 1:
      if key not in list(k_gram_model[2].keys()):
        return [(calculate_char_probability(query), query)]

    candidate_tokens = []
    if length < 8:                   
      k = length+1
    else: 
      k = 9

    while(len(key)>k-1):
      if isinstance(key,list):
        key.pop(0)
      else:
        break

    # going for smaller k grams if key not found in bigger one 
    while(k>1):
        if k>2 and tuple(key) not in list(k_gram_model[k].keys()):
          key.pop(0)
          k -= 1
        elif k==2:
          if key in list(k_gram_model[k].keys()):
            candidate_dict = k_gram_model[k][key]
            break
          else:
            k -= 1
        else:
          candidate_dict = k_gram_model[k][tuple(key)]
          break

    if k==1:
      return [(calculate_char_probability(query), query)]
    
    #base case: empty candidate_dict 
    if bool(candidate_dict) == False:
      return [(calculate_char_probability(query), query)]

    resultant_queries = []
    for i in candidate_dict.keys():
      if candidate_dict[i] > threshold:
        new_query = query + ' ' + i
        resultant_queries += query_prediction(new_query, k_gram_model, threshold, True)
    
    return Sort_Tuple(resultant_queries)[0:10]


In [ ]:
#The function query_prediction() uses only the word-gram model
#This function is further called in final_pred() coded below

print(query_prediction('www', k_gram_model, 0, False))
print(query_prediction('hampt', k_gram_model, 0, False))
print(query_prediction('weddings', k_gram_model, 0.25, True))
print(query_prediction('uniq', k_gram_model, 0.25, False))

[(69305.96973388329, 'www my $'), (33717.71513841273, 'www mys $'), (17314.080336879633, 'www mas $'), (13157.394187558324, 'www e $'), (6502.942835987436, 'www mr $'), (4538.681630739294, 'www g4 $'), (4470.236697568606, 'www mtv $'), (2157.001730532563, 'www eb $'), (1673.1966801772867, 'www jobs com $'), (1368.4686770285891, 'www das $')]
[(0.006182789966610348, 'hampton house $'), (4.277629755050705e-08, 'hamptonmaidinn com $'), (1.3422695697602991e-10, 'hampton hotel york pa $'), (1.3764293929755488e-23, 'hampton beach nh chamber of commerce $')]
[(6.6080187974016625e-09, 'weddings las vegas $'), (9.98337643638317e-19, 'weddings in central park new york $')]
[(0.011935343850234863, 'unique marine $'), (0.010927045196449292, 'unique thongs $'), (1.2090403337904697e-11, 'unique vinyl siding $')]


In [ ]:
#calculation of Perplexity to quantify the model
def find_perplexity(resultant_query):
  # print(resultant_query)
  # print(type(resultant_query[1]))
  prob=calculate_char_probability(resultant_query[1])
  return prob**((-1)/len(resultant_query))  

#Final Prediction of The Query

In [ ]:
#first complete the last word using char gram model till we get space or $
#then we check for word gram model
#if that doesnt work, then use char gram model
#in case of exception, try using edit distance

def check_for_word_gram(query,k_gram_model):
  query_words=query.split()
  # print(query_words)
  
  while len(query_words):
    # print(len(query_words))
    if (len(query_words)==1):
      if(query_words[0] in k_gram_model[2].keys()):
        # print('word=', query_words[0])
        return True
      else:
        return False

    elif len(query_words)+1 < 10 and tuple(query_words) in k_gram_model[len(query_words)+1].keys():
      return True
    else:
      query_words.pop(0)
    
  return False  


#final prediction of a query
def final_pred(query):
  temp_query = ""
  for i in query:
    if i.isalpha() or i==" ":
      temp_query += i

  query = temp_query.strip(" ")
  if(len(query) == 0):
    return []

  final_ans = []
  candidate_queries=[]

  flag=True
  tokens = query.split()
  length = len(tokens)
  # print('tokens=', tokens)
  x = mapping_from_right(list(tokens_count.keys()))
  possible_words = []
  if tokens[length-1] in x.keys():
    possible_words = mapping_from_right(list(tokens_count.keys()))[tokens[length-1]]

  # print('possible words1=',possible_words)
  if length> 1:
    if tokens[length-2] in k_gram_model[2].keys():
      bi_words = list(k_gram_model[2][tokens[length-2]].keys())
      possible_words = intersection(possible_words, bi_words)
    # print('pos=', possible_words)
    else:
      possible_words = []

  # print('possible words2=',possible_words)
  resultant_queries = []

  if len(possible_words):
    flag = False
    
  for i in possible_words:
    tokens[length-1] = i
    new_query = ''
    for t in tokens:
      new_query += (t+' ')
    new_query = new_query[:-1]
    resultant_queries += query_prediction(new_query, k_gram_model, 0, True)
    
  if flag==False:
    # return Sort_Tuple(resultant_queries)[0:10]
    final_ans += Sort_Tuple(resultant_queries)[0:10]
      
  else:    #means now complete the last word with char gram model
    # char prediction - flag == true
    l1=char_prediction(query,k_gram_model_char, True)
    for tup in l1:
      if(tup[1][len(tup[1])-1]=='$'):
        final_ans.append(tup)
      else:
        candidate_queries.append(tup[1])
  
  for q in candidate_queries:
    if (check_for_word_gram(q, k_gram_model)):
      ans=query_prediction(q,k_gram_model,0,False)
      final_ans+=ans
    else:
      ans=char_prediction(q,k_gram_model_char,False)
      final_ans+=ans
  
  return Sort_Tuple(final_ans)[0:10]


#Example Runs of Advanced model and baseline model

In [ ]:
#advanced model
final_pred("hamp")

[(0.006182789966610348, 'hampton house $'),
 (4.277629755050705e-08, 'hamptonmaidinn com $'),
 (1.3422695697602991e-10, 'hampton hotel york pa $'),
 (1.3764293929755488e-23, 'hampton beach nh chamber of commerce $')]

In [ ]:
#baseline model used for comparision
char_prediction_trigram("hamp",k_gram_model_char,False)

[(47700.60894018203, 'hamp $'),
 (33403.384630004606, 'hamps $'),
 (9838.50260473746, 'hamp3 $'),
 (7348.6023751243265, 'hample $'),
 (5201.339773675949, 'hampa $'),
 (1044.6454083666958, 'hampa com $'),
 (186.33537477714947, 'hampion $'),
 (157.52613908431783, 'hampoo $'),
 (6.889626245459942, 'hampaign $'),
 (3.9371907209985526, 'hamp parts $')]

In [ ]:
#advanced model
final_pred("time")

[(435177.5374343186, 'times $'),
 (379109.89080131997, 'time $'),
 (0.17840196797098584, 'time 5 4 06 $'),
 (0.004731833469136794, 'time results $'),
 (0.0020234611464014095, 'time slogan $'),
 (0.0008440999123392078, 'timemagazine com $'),
 (1.9043097285251848e-06, 'times picayune $'),
 (9.702639727348005e-07, 'time to get over $'),
 (5.516976911323449e-08, 'time out magazine $'),
 (1.5283901305078644e-08, 'timewarnercable $')]

In [ ]:
#baseline model used for comparision
char_prediction_trigram("time",k_gram_model_char,False)

[(435177.5374343186, 'times $'),
 (379109.89080131997, 'time $'),
 (42620.59683090194, 'times com $'),
 (2142.400624236118, 'timeo $'),
 (1760.6830224062755, 'timez com $'),
 (650.6732936318786, 'timeals $'),
 (472.16449030129365, 'timeters $'),
 (141.61220540853986, 'times org $'),
 (121.22413044259272, 'timeet com $'),
 (38.780807696576, 'timepage $')]

#Running final_pred() on test data



In [ ]:
test_data = test.loc[:, 0]
# # print(test_data)
j=0
i=0

while(j<len(test_data)):
  try:
    for i in range(j, len(test_data)):
      print("New Query Start: ", end=" ")
      print(test_data[i])
      # preprocess the query
      # q_final=query_correction(test_data[i], vocab, bigram_model, 2, 0.5)
      pred=final_pred(test_data[i])
      for predictions in pred:
        print("Prediction:    ", predictions)
        print(find_perplexity(predictions))
      print("/////////////////\n")
    
  except Exception as e:
    j=i
    try:
      #correcting the query
      q_final_list=query_correction(test_data[i],vocab,bigram_model,2,0.5)
      q_final=""
      for i in range(1,len(q_final_list)-1):
        q_final+=q_final_list[i]
      pred=char_prediction_trigram(q_final, k_gram_model_char, True)
      for predictions in pred:
        print("Prediction:    ", predictions)
        print(find_perplexity(predictions))
        print("/////////////////////\n")
    except Exception as e2:
      print("Erroreneous query=", test_data[j])
      print("Query No.=", j)
      print("Exception = ", e)

    j+=1
  